# Qwen3-1.7B Multi-turn Dialogue Full SFT (MPS Optimized)

針對 Apple Silicon MPS 優化的 Qwen3-1.7B 全參數微調訓練

In [ ]:
# 安裝必要套件
!pip install transformers>=4.45.0 datasets accelerate trl -q

In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)

# MPS 設定
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ 使用 MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print("⚠️ MPS 不可用，使用 CPU")

# 環境變數優化
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # 防止 MPS OOM
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# 模型與資料集路徑
MODEL_NAME = "Qwen/Qwen3-1.7B"
DATASET_PATH = "dataset/coffee_order_zhtw.jsonl"
OUTPUT_DIR = "./qwen3_coffee_sft"

# 訓練超參數 (針對 MPS 全參數 SFT 優化)
MAX_LENGTH = 512
BATCH_SIZE = 1  # MPS 記憶體有限
GRADIENT_ACCUMULATION = 16  # 全參數訓練需要更多累積
LEARNING_RATE = 5e-6  # 全參數 SFT 用較小 lr
NUM_EPOCHS = 3

In [ ]:
# 載入資料集
def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

raw_data = load_jsonl(DATASET_PATH)
print(f"載入 {len(raw_data)} 筆對話資料")

In [ ]:
# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

In [ ]:
# 將對話轉換為訓練格式
def format_conversation(example):
    """將 multi-turn 對話轉換為 Qwen chat 格式"""
    conversations = example["conversations"]
    
    messages = []
    for turn in conversations:
        role = turn["role"]
        content = turn["content"]
        messages.append({"role": role, "content": content})
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

formatted_data = [format_conversation({"conversations": d["conversations"]}) for d in raw_data]
dataset = Dataset.from_list(formatted_data)

print(f"範例對話:\n{formatted_data[0]['text'][:500]}...")

In [ ]:
# Tokenize 資料集
def tokenize_function(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
        return_tensors=None
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"訓練集: {len(train_dataset)} 筆")
print(f"驗證集: {len(eval_dataset)} 筆")

In [ ]:
# 載入模型 (MPS 優化)
print("載入模型中...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.float32,  # 全參數 SFT 建議用 float32 穩定性較好
    device_map=None,
    low_cpu_mem_usage=True,
)

model = model.to(device)
model.gradient_checkpointing_enable()  # 節省記憶體

# 確保所有參數可訓練
for param in model.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"模型已載入至 {device}")
print(f"總參數: {total_params:,}")
print(f"可訓練參數: {trainable_params:,} (100%)")

In [ ]:
# 訓練參數 (MPS 全參數 SFT 優化)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,
    bf16=False,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_torch",
    max_grad_norm=1.0,  # 梯度裁剪防止爆炸
)

In [ ]:
# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

# 建立 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# 開始訓練
print("🚀 開始全參數 SFT 訓練...")
trainer.train()

In [ ]:
# 儲存模型
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ 模型已儲存至 {OUTPUT_DIR}")

In [ ]:
# 測試推論
model.eval()

test_messages = [
    {"role": "system", "content": "你是一位專業的咖啡點餐助理，負責協助使用者完成點餐。菜單包含：美式、拿鐵、燕麥奶拿鐵、鮮奶。"},
    {"role": "user", "content": "我想要一杯冰拿鐵"}
]

input_text = tokenizer.apply_chat_template(
    test_messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"模型回應:\n{response}")